#**Project Omega**

#Project's Goal
The main research goal is to set a program that will predict the gender of the author of a tweet and we could extend the model to any text.<br><br>
To do so, we will use some text analytics techniques seen in class. Those techniques are tokenization, BOW (bag of words), TF-IDF and stylometrie.<br><br>
In term of business, with this idea we aim to enhence marketing targeting. Third party advertisement companies can have better knowledge on Twitter’s gender distribution. Thus, they can advise companies that want to make advertisement on social network like Twitter. 


##Importation and Installation of methods
For this project, we will use different packages. The main part will be about text mining. For this, we will use packages seen in class, which are *spacy*, *nltk* and *enchant*. 

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
import enchant
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
!pip install spacy
!apt install -qq enchant
!pip install pyenchant
!pip install nltk
!python -m spacy download en
nltk.download('punkt')
nltk.download('wordnet')

enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

##Data Cleaning <br>
We first began by separate tweet by gender. We made new tables, one for male, one for female, one for brand and one with the unknown gender. <br>
We are not interested by all the feautres (columns), we kept the "gender", the "text" and the "name" for the brand. <br>
We chose to split the data by gender to be able to construct general BOW for each gender.

In [0]:
#Import data
data = pd.read_csv('https://raw.githubusercontent.com/XaviJunior/omega/Data/Data/gender-classifier-DFE-791531.csv',encoding="latin-1")
textgender=data[["gender","text"]]

#New Table with Male Only and Split in Test/Train Data
Male = textgender[textgender['gender'] == 'male']
Maletext=Male["text"]
XM=Maletext
ym=Male["gender"]
XM_train, XM_test, ym_train, ym_test = train_test_split(XM, ym, test_size=0.2, random_state=10)
ListMale=XM_train.values.tolist()

#New Table with Female Only and split in Test/Train Data
Female = textgender[textgender['gender'] == 'female']
Femaletext=Female["text"]
XF=Femaletext
yf=Female["gender"]
XF_train, XF_test, yf_train, yf_test = train_test_split(XF, yf, test_size=0.2, random_state=10)
ListFemale=XF_train.values.tolist()

#New Table with Unknown Only
Unknown=textgender[textgender['gender'] == 'unknown']
Unktext=Unknown["text"]
ListUnk=Unktext.values.tolist()

#New Table with Brand Only
Brand = data[data['gender'] == 'brand']
Brandtext=Brand[["text"]]
ListBrand=Brandtext.values.tolist()

###Merging
Now that we create different table for both gender. We will merge all the tweets in two documents, for one male and one for female.

In [0]:
#Merge all Male Text in a single string
TextMale=" "
for i in range(0,len(ListMale)):
  TextMale=TextMale+ListMale[i]
textmale=TextMale.lower()
#Merge all Female Text in a single string
TextFemale=" "
for i in range(0,len(ListFemale)):
  TextFemale=TextFemale+ListFemale[i]
textfemale=TextFemale.lower()
textfemale

' #mcm goes out to my one and only \x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f @realeliascarr@gelaaneybug no silly, to sit down and think that you can beat these downer thoughts.un agency links hot dogs and other processed meat to cancer - new york times: timeun agency links hot dogs and... https://t.co/mbt1azjthethat makes no sense but i cba to tell the whole story lmaothe history of the hmmwv (humvee) has ties with jeep. #watchesyou made me ask myself why everyday after you stabbed me in the back but now i know why... you were a selfish ass named tord.me and my halloween crew https://t.co/boq5coengdhow the hell i am supposed to write a 1,500 word essay about one primary and one secondary source? #historyprobs@emilycanham you don\'t bc your hair is lovely and i love it soenter to #win a $100 #amazon #giftcard #giveaway #free #giveaways #sweeps #nutritionmatters #ad  https://t.co/axx6gegdnqhi princess of light eyes _ùõð. \nyou and a great makeup artist _ùõã_ùõü .\nyou are aw

###Tokenization
Then with those two documents, we have to do some tokenization. The tokenization will allow us to separate each word and then to be able to count the frequencie of each single word. The tokenization's technique chosen is the white-space one.

In [0]:
# Word tokenization


# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

#  "nlp" Object is used to create documents with linguistic annotations.
Doc_F = nlp(textfemale)
Doc_M=nlp(textmale)

# Create list of word tokens for female text
token_F = []
for token in Doc_F:
    token_F.append(token.text)
print(token_F)

# Create list of word tokens for female text
token_M = []
for token in Doc_M:
    token_M.append(token.text)
print(token_M)

[' ', '#', 'mcm', 'goes', 'out', 'to', 'my', 'one', 'and', 'only', '\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f', '@realeliascarr@gelaaneybug', 'no', 'silly', ',', 'to', 'sit', 'down', 'and', 'think', 'that', 'you', 'can', 'beat', 'these', 'downer', 'thoughts.un', 'agency', 'links', 'hot', 'dogs', 'and', 'other', 'processed', 'meat', 'to', 'cancer', '-', 'new', 'york', 'times', ':', 'timeun', 'agency', 'links', 'hot', 'dogs', 'and', '...', 'https://t.co/mbt1azjthethat', 'makes', 'no', 'sense', 'but', 'i', 'cba', 'to', 'tell', 'the', 'whole', 'story', 'lmaothe', 'history', 'of', 'the', 'hmmwv', '(', 'humvee', ')', 'has', 'ties', 'with', 'jeep', '.', '#', 'watchesyou', 'made', 'me', 'ask', 'myself', 'why', 'everyday', 'after', 'you', 'stabbed', 'me', 'in', 'the', 'back', 'but', 'now', 'i', 'know', 'why', '...', 'you', 'were', 'a', 'selfish', 'ass', 'named', 'tord.me', 'and', 'my', 'halloween', 'crew', 'https://t.co/boq5coengdhow', 'the', 'hell', 'i', 'am', 'supposed', 'to', 'w

###Cleaning
Now that we have tokens, we to clean the lists by removing stop words and other token such as "\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f\x89\x9d_\x95ü\x8f". To do so, we will use a token list available in *spacy* and filter words that are not in the english dictionnary available on *pyenchant*. We know that by using an ensglish dictionnary we may lose some expression that are used on twitter. But for us, it was the only sustainable solution to filter non-sense tokens.

In [0]:
#Implementation of stop words:
Words_F=[]
d = enchant.Dict("en_US")

#filtering stop words , words not in english dictionnary and punctuation for female text and create a list
TextF=''
for i in token_F:
  if d.check(i)==True:
    TextF+=' '+i
doc=nlp(TextF)
for word in doc:
  if word.is_stop==False and word.is_punct==False:
    Words_F.append(word)

#change the type of list's element
words_f=[]
for j in Words_F:
  o=str(j)
  lemm=lemmatizer.lemmatize(o)
  words_f.append(lemm)

In [0]:
#Implementation of stop words:
Words_M=[]
TextM=''
#filtering stop words , words not in english dictionnary and punctuation for female text and create a list
for i in token_M:
  if d.check(i)==True:
    TextM+=' '+i
doc=nlp(TextM)
for word in doc:
  if word.is_stop==False and word.is_punct==False:
    Words_M.append(word)
#change the type of list's element
words_m=[]
for j in Words_M:
  p=str(j)
  lemma=lemmatizer.lemmatize(p)
  words_m.append(lemma)

##Bag Of Words

In [0]:
Fem=' '.join(words_f)
Mal=' '.join(words_m)
text=[Fem,Mal]
count = CountVectorizer()
bow = count.fit_transform(text)

In [0]:
feature_names = count.get_feature_names()
pd.DataFrame(bow.todense(),columns=feature_names, index=['Female','Male'])

,00,000,00000000000000002344,007,01,044,05,06,07,07840458711,09,10,100,1000,100th,103,105,107,108,1091,10th,11,110,1115,114,115,116,11th,12,120,120315,121,13,138,13th,14,140,1408,144,14th,...,yawn,ye,yea,yeah,year,yell,yelled,yelling,yellow,yelp,yep,yes,yesterday,yield,yikes,yo,yoga,yogurt,young,younger,yous,youth,yr,yummy,yup,zap,zen,zenith,zero,zillion,zip,zipped,zodiac,zombie,zone,zoning,zonked,zoo,zoom,zooming
Female,2,5,1,0,1,1,1,1,0,1,1,32,7,3,1,0,0,2,1,1,0,10,0,0,0,0,0,1,15,2,0,1,4,0,0,2,0,0,1,0,...,0,0,2,29,104,1,3,1,3,0,5,36,15,0,1,6,2,2,5,3,1,1,5,2,1,0,2,1,2,1,0,1,0,4,0,0,1,1,1,1
Male,1,12,0,1,0,0,0,1,2,0,0,36,19,4,0,1,1,0,0,0,2,8,2,1,1,1,1,0,9,0,1,1,5,2,3,5,2,2,0,1,...,1,1,5,26,103,1,2,0,1,1,6,36,10,1,0,19,0,0,18,2,0,4,8,1,2,1,0,0,2,0,1,0,1,7,2,1,0,0,0,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1, 1))
features = tfidf.fit_transform(text)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names(),
    index=['Female','Male'])

,00,000,00000000000000002344,007,01,044,05,06,07,07840458711,09,10,100,1000,100th,103,105,107,108,1091,10th,11,110,1115,114,115,116,11th,12,120,120315,121,13,138,13th,14,140,1408,144,14th,...,yawn,ye,yea,yeah,year,yell,yelled,yelling,yellow,yelp,yep,yes,yesterday,yield,yikes,yo,yoga,yogurt,young,younger,yous,youth,yr,yummy,yup,zap,zen,zenith,zero,zillion,zip,zipped,zodiac,zombie,zone,zoning,zonked,zoo,zoom,zooming
Female,0.001907,0.004768,0.00134,0.000000,0.00134,0.00134,0.00134,0.000954,0.000000,0.00134,0.00134,0.030516,0.006675,0.002861,0.00134,0.000000,0.000000,0.002681,0.00134,0.00134,0.000000,0.009536,0.000000,0.000000,0.000000,0.000000,0.000000,0.00134,0.014304,0.002681,0.000000,0.000954,0.003814,0.000000,0.000000,0.001907,0.000000,0.000000,0.00134,0.000000,...,0.000000,0.000000,0.001907,0.027655,0.099175,0.000954,0.002861,0.00134,0.002861,0.000000,0.004768,0.034330,0.014304,0.000000,0.00134,0.005722,0.002681,0.002681,0.004768,0.002861,0.00134,0.000954,0.004768,0.001907,0.000954,0.000000,0.002681,0.00134,0.001907,0.00134,0.000000,0.00134,0.000000,0.003814,0.000000,0.000000,0.00134,0.00134,0.00134,0.00134
Male,0.001111,0.013329,0.00000,0.001561,0.00000,0.00000,0.00000,0.001111,0.003122,0.00000,0.00000,0.039986,0.021104,0.004443,0.00000,0.001561,0.001561,0.000000,0.00000,0.00000,0.003122,0.008886,0.003122,0.001561,0.001561,0.001561,0.001561,0.00000,0.009997,0.000000,0.001561,0.001111,0.005554,0.003122,0.004683,0.005554,0.003122,0.003122,0.00000,0.001561,...,0.001561,0.001561,0.005554,0.028879,0.114405,0.001111,0.002221,0.00000,0.001111,0.001561,0.006664,0.039986,0.011107,0.001561,0.00000,0.021104,0.000000,0.000000,0.019993,0.002221,0.00000,0.004443,0.008886,0.001111,0.002221,0.001561,0.000000,0.00000,0.002221,0.00000,0.001561,0.00000,0.001561,0.007775,0.003122,0.001561,0.00000,0.00000,0.00000,0.00000


##Cleaning of the Test Data

In [0]:
TEST=XM_test.values.tolist()
a=[]
aa=[]
Wordsmm=[]
for i in TEST:
  j=i.split(" ")
  a.append(j)

for i in a:
  testtext=''
  for p in i:
    if p:      
      if d.check(p)==True:
        testtext+=' '+p
  aa.append(testtext)


ret=[]
for sentences in aa:
  Wordsmm=''
  sentences=sentences.lower()
  sentences=nlp(sentences)
  for w in sentences:
    if w.is_stop==False and w.is_punct==False:
      w=str(w)
      Wordsmm+=' '+w
  ret.append(Wordsmm)


In [0]:
ret

['   brady rodgers shown body language cam great leaders cam immature',
 '   media stock green ticker',
 '   maybe kind feel way',
 '   new',
 '   dumb levels women wear burkas knock outfits',
 '   tidal raped shit american oxygen music took away',
 '   bleeding heart welcomes sharks',
 '   house fantastic news country thank',
 '   save date change actors',
 '   okay buy cookies tea got ta stuffing couch use',
 '   day 42 seeing glad',
 '   saw mr fox texting driving',
 '   administration hard working committed busy touring world voted tourist government',
 '   cobblers chief exec james whiting revealed club owes taxman',
 '   says natural going',
 '   5 game want online experience works',
 '   strive greatness persistent',
 '   wanted trump donate',
 '   new dusk till scares season',
 '   big insane race',
 '   wow coat',
 '   dark horse ditches queen st 401 work play 401 richmond soon',
 '   influence weather significant factor case prolonged',
 '   age social media trolls people for

In [0]:
#text=[Fem,Mal,aa[100]]
#count = CountVectorizer()
#bow = count.fit_transform(text)
#feature_names = count.get_feature_names()
#pd.DataFrame(bow.todense(),columns=feature_names, index=['Female','Male','Test'])
#cosine_similarity(bow[2],bow)